@article{wagner2022dawn,
  title={Dawn of the transformer era in speech emotion recognition: closing the valence gap},
  author={Wagner, Johannes and Triantafyllopoulos, Andreas and Wierstorf, Hagen and Schmitt, Maximilian and Burkhardt, Felix and Eyben, Florian and Schuller, Bj{\"o}rn W.},
  journal={arXiv preprint arXiv:2203.07378},
  year={2022}
}

In [11]:
import audeer
import audonnx
import numpy as np

url = 'https://zenodo.org/record/6221127/files/w2v2-L-robust-12.6bc4a7fd-1.1.0.zip'
cache_root = audeer.mkdir('cache')
model_root = audeer.mkdir('model')

archive_path = audeer.download_url(url, cache_root, verbose=True)
audeer.extract_archive(archive_path, model_root)
model = audonnx.load(model_root)

sampling_rate = 16000
signal = np.random.normal(size=sampling_rate).astype(np.float32)
model(signal, sampling_rate)


{'hidden_states': array([[-0.00716192,  0.00631102, -0.00802942, ...,  0.00670052,
          0.00957113,  0.00271725]], dtype=float32),
 'logits': array([[0.6756788, 0.647147 , 0.5066436]], dtype=float32)}

In [31]:
signal

array([ 0.35366887, -1.1935568 , -0.1309103 , ..., -2.6430488 ,
       -0.11577458,  0.20400122], dtype=float32)

In [1]:
from datasets import load_dataset
import utils.audio_dataset_utils as audioUtils
import pandas as pd

test_dataset = load_dataset("csv", data_files={"validation": "content/data/test.csv"}, delimiter="\t")["validation"]

Using custom data configuration default-514e4f917f0929bb
Found cached dataset csv (C:/Users/tonib/.cache/huggingface/datasets/csv/default-514e4f917f0929bb/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
def preprocess_function(examples):
    speech_list = [audioUtils.speech_file_to_array_fn(path, 16000) for path in examples["path"]]
    result = speech_list

    return result

In [77]:
import librosa
x = audioUtils.speech_file_to_array_librosa(test_dataset["path"][4], 16000)
i=123
y,sr = librosa.load(test_dataset["path"][i], sr=16000)
print(test_dataset["emotion"][i])

happy


In [78]:
#arousal, dominance, valence.
model(y, 16000)

{'hidden_states': array([[-0.00796949,  0.00740917, -0.01084459, ...,  0.0075842 ,
          0.0091475 ,  0.00472357]], dtype=float32),
 'logits': array([[0.59236515, 0.62931955, 0.43607527]], dtype=float32)}

In [7]:
test_dataset = test_dataset.map(
    preprocess_function,
    batch_size=50,
    batched=True
    #num_proc=4
)

NameError: name 'preprocess_function' is not defined